In [1]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
# %cd /content/drive/MyDrive/
# !kaggle datasets download -d ashery/chexpert -p chexpert_data

In [4]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [5]:
class CheXpertDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):

        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # مسیر نسبی تصویر از ستون CSV
        img_rel_path = self.labels_df.iloc[idx]['Path']

        # حذف پیشوند "CheXpert-v1.0-small" اگر در مسیر بود
        if img_rel_path.startswith("CheXpert-v1.0-small"):
            img_rel_path = img_rel_path[len("CheXpert-v1.0-small")+1:]  # +1 برای حذف اسلش بعدی

        # مسیر کامل تصویر با join کردن مسیر ریشه و مسیر نسبی اصلاح شده
        img_path = os.path.join(self.img_dir, img_rel_path)

        # بارگذاری تصویر با تبدیل به RGB (3 کاناله)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"[WARNING] Could not load image: {img_path} -- {e}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        if self.transform:
            image = self.transform(image)

        # گرفتن لیبل‌ها و infer_objects برای جلوگیری از warning
        labels = self.labels_df.iloc[idx][['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']]
        labels = labels.infer_objects(copy=False).fillna(0).values.astype('float32')

        return image, labels



In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [28]:
base_csv_file='/content/drive/MyDrive/chexpert_data_v2/train.csv'
df = pd.read_csv(base_csv_file)


In [29]:
df_subset = df.sample(frac=0.3, random_state=42).reset_index(drop=True)


In [30]:
df_subset.to_csv("chexpert_30percent.csv", index=False)


In [31]:
train_dataset = CheXpertDataset(
    csv_file="chexpert_30percent.csv",
    img_dir='/content/drive/MyDrive/chexpert_data_v2/',
    transform=transform
)

In [32]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)


In [33]:
val_dataset = CheXpertDataset(
    csv_file='/content/drive/MyDrive/chexpert_data_v2/valid.csv',
    img_dir='/content/drive/MyDrive/chexpert_data_v2/',
    transform=transform
)

In [34]:
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [35]:
model = model.to(device)

In [36]:
model.head = nn.Linear(model.head.in_features, 5)
model = model.to(device)

In [37]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [26]:
import gc

# del images, labels, outputs, loss
gc.collect()
torch.cuda.empty_cache()


In [38]:

num_epochs = 5


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # tqdm loop برای هر batch
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=True)

    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # جمع کردن loss برای محاسبه میانگین
        running_loss += loss.item() * images.size(0)

        # آپدیت نوار پیشرفت
        loop.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"\n✅ Epoch {epoch+1}/{num_epochs} - Avg Loss: {epoch_loss:.4f}\n")


Epoch [1/5]:   2%|▏         | 24/1048 [12:55<7:40:33, 26.99s/it, loss=0.0806] Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ffb34008fe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Epoch [1/5]:   2%|▏         | 26/1048 [13:58<7:32:48, 26.58s/it, loss=-0.167] Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ffb34008fe0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data

KeyboardInterrupt: 

In [19]:
import os
print(os.path.exists('/content/drive/MyDrive/chexpert_data_v2/train/patient20372/study15/view1_frontal.jpg'))


True


In [ ]:
/CheXpert-v1.0-small/train